## CS 5010 Final Project
Date: 7 August 2020

Project members: 

Aubrey Brockmiller (alb3cb)

Hemani Choksi (hc8nd)

Bryan DeMarcy (bd2qn)

Mike Wetklow (mw8hu)

### Data Sets

Below are the data sources we used for our project.

Census population data

Source:https://www.census.gov/data/tables/time-series/demo/popest/2010s-state-total.html

List of Us Governors

Source:https://en.m.wikipedia.org/wiki/List_of_United_States_governors

State Abbreviations

Source: https://worldpopulationreview.com/states/state-abbreviations

HHS Spending Data

Source: https://taggs.hhs.gov/coronavirus

COVID-19 Data (**Note:** The COVID-19 data set ranges from the start of the pandemic up to June 30th, 2020)

Source: https://covidtracking.com/data/download

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plot
from functools import reduce
import plotly.graph_objects as go
import folium
import os
import pygal
from pygal.style import Style

### 1. Data Wrangling

In [ ]:
#Read the csv files into pandas
population = pd.read_csv(r'rawData/populationData.csv')
spendingHHS = pd.read_csv(r'rawData/HHSspending.csv')
governors = pd.read_csv (r'rawData/govAffiliation.csv')
abbreviations = pd.read_csv(r'rawData/stateAbbreviations.csv')
covid = pd.read_csv(r'rawData/COVIDdata.csv')

In [ ]:
#Reduce the dataframes to only the columns we need for our analysis
population = pd.DataFrame(population, columns= ['NAME', 'POPESTIMATE2019'])
abbreviations = pd.DataFrame(abbreviations, columns = ['State', 'Code'])
covid = pd.DataFrame(covid, columns = ['date', 'state', 'positive', 
                                       'hospitalizedCurrently', 'hospitalizedCumulative', 
                                       'recovered', 'death', 'positiveIncrease', 'totalTestResults'])

In [ ]:
#Rename columns to get rid of the multiple variations of the same column and to be more descriptive
population = population.rename(columns={'NAME': 'State', 'POPESTIMATE2019': 'population2019'})
spendingHHS = spendingHHS.rename(columns={'State':'stateCode', 'Award_Amount':'Amount'})
governors = governors.rename(columns={'state': 'stateCode'})
abbreviations = abbreviations.rename(columns={'Code': 'stateCode'})
covid = covid.rename(columns={'state': 'stateCode'})

In [ ]:
#Reformat the date column in the covid dataframe and sort by date
covid['date'] = pd.to_datetime(covid['date'].astype(str), format='%Y%m%d')

#Use grouping to find the total HHS spending by state and save it as a dataframe
#The dataframe was given a new name so we could use the spendingHHS dataframe again in Section 4
funding = pd.concat([spendingHHS.groupby(['stateCode'])['Amount'].sum()], axis=1)

In [ ]:
#Merge the population and abbreviations dataframes so population contains state names
population = pd.merge(left=abbreviations, right=population, left_on='State', right_on='State')

#Create a list of dataframes to merge
data_frames = [governors, population, funding]

#Merge the list of dataframes
df_merged = reduce(lambda left,right: pd.merge(left,right,on=['stateCode']), data_frames)

#Merge the abbreviations and COVID dataframes so the COVID data contains state names and sort by date
covid = pd.merge(left=abbreviations, right=covid, left_on='stateCode', right_on='stateCode')
covid = covid.sort_values(by=['date'], ascending=True)

In [ ]:
#Check dataframes by printing them to csv files
df_merged.to_csv(r'tidyData/df_merged.csv', index=False)
covid.to_csv(r'tidyData/covid.csv', index=False)

### 2. Overall Analysis on the 50 US States

This set of code looks at HHS spending and COVID-19 data for the entire United States.

In [ ]:
#lets print the States where they spent more than $150 per person 
df_merged['perCap']=(df_merged['Amount'] / df_merged['population2019']).round(decimals=2)
print(df_merged.loc[df_merged['perCap'] > 150].sort_values(by=['perCap'],ascending=False).to_string(index=False))

In [ ]:
#printing states that spent less than $55 per person
print(df_merged.loc[df_merged['perCap'] < 55].sort_values(by=['perCap'],ascending=True).to_string(index=False))

In [ ]:
#now lets print states that have the highest amount of recoveries
print(covid.loc[(covid['date'] == '6/30/2020') & (covid['recovered'] > 70000)])

In [ ]:
#print states that currently have less than 100 people hospitalized
covid.loc[(covid['date'] == '6/30/2020') & (covid['hospitalizedCurrently']<100)] \
    .sort_values(by=['hospitalizedCurrently'],ascending=True)

In [ ]:
#print states that haven't had any new positive cases in the past week
covid.loc[(covid['date'] > '6/23/2020') & (covid['date'] <= '6/30/2020') \
    & (covid['positiveIncrease'] == 0)]

In [ ]:
#Choropleth map of the US based on HHS spending

#Create a copy of our df_merged dataframe so we can convert everything in the copy to a string
df_mergedCopy = df_merged.copy()

#Convert columns to strings
for col in df_mergedCopy.columns:
    df_mergedCopy[col] = df_mergedCopy[col].astype(str)

#Text that will be displayed when hovering over each state
df_mergedCopy['text'] = df_mergedCopy['stateCode'] + '<br>' + \
    'Governor Affiliation: ' + df_mergedCopy['govAffiliation']  + '<br>' + \
    'Governor: ' + df_mergedCopy['govName'] + '<br>' + \
    'Population: ' + df_mergedCopy['population2019'] + '<br>' + \
    'Money per Person: $ ' + df_mergedCopy['perCap']

fig = go.Figure(data=go.Choropleth(
    locations=df_mergedCopy['stateCode'], # Spatial coordinates
    z = df_mergedCopy['Amount'].astype(float), # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    text=df_mergedCopy['text'], # hover text
    colorscale = 'Greens',
    colorbar_title = "Millions USD",
))

fig.update_layout(
    title_text = 'COVID-19 HHS Spending',
    geo_scope='usa', # limit map scope to USA
)

fig.show()

### 3. User Input Query and Analysis

This next section allows the user to input a list of states for comparative analysis. Only states from the user input will be shown so HHS spending and COVID-19 data can be compared between states.

**Note:** The layout and figure sizes were set to only compare a handful of states at once. The user can input "all" to compare all 50 states simultaneously; however, the visuals will be very crowded and difficult to read.

In [ ]:
#Set the index in the df_merged dataframe for lookup purposes
df_merged = df_merged.set_index('stateCode')

In [ ]:
#This class will be used for data visualizations and user input
class Data:
    
    def __init__(self, states=None): #Constructor
        self.df = pd.DataFrame() #Create an empty dataframe for each instance
        if states is None: 
            #Ask user for a list of state abbreviations or 'all' to represent all 50 states
            self.states = input("Enter a list of state abbreviations separated by a space or type 'all' without quotes to compare all 50 states: ")
        elif states == False: #This if statement allows us to create an empty dataframe later without being prompted for input
            self.states = [] 
        else: #This allows the user to input states as a list when creating the instance; however, we won't use it this way
            self.states = states
        
    def userStates(self): #Method for user input
        #Make the userList global so we can call it later
        global userList
        
        #If the user inputs "all", the list will include all state abbreviations
        if self.states == "all":
            userList = df_merged.index.values.tolist()
            print("Thank you for your input. We will show you the data for all 50 states!")
        
        #If the user does not input "all", split the input by space
        else:
            userList = self.states.split()
            try:
                test = userList[0] #Test to see if the list is empty
                for state in userList:
                    testList = []
                    testList.append(df_merged.loc[state].values[0:(len(df_merged.columns)+1)]) #Search the df_merged dataframe and append every row based on the state in the user list
                print("Thank you for your input. We will show you the data for the states listed below. \n", userList)        
            
            except IndexError: #Prints an error if the user enters an empty list
                return print("ERROR: You did not enter a state abbreviation. Please rerun the code using at least one of the state abbreviations below. \n", df_merged[["State"]],"\n")
                
            except KeyError: #Prints an error if the user does not enter a valid state abbreviation
                return print("ERROR: ",str(state)," is not one of the state abbreviations. Please rerun the code using any combination of the state abbreviations below. \n", df_merged[["State"]],"\n")
                        
    #This method is used to create our line, bar, and pie charts
    #Required arguments are ones that appear in all plot types, otherwise they are optional.
    def userPlot(self, plotType, title, y, ylabel, legendIndex, x=None, xlabel=None, 
                 axis=None, legendlabel=None, size=None, rotation=None):
        
        if plotType == "line": #Create a line plot using the specified dataframe
            self.df.plot(kind=plotType, x=x, y=y, ax=axes[axis], legend=legendIndex, 
                         label=self.df.iloc[0][legendlabel])
            axes[axis].xaxis.set_label_text(xlabel)
            axes[axis].yaxis.set_label_text(ylabel)
            axes[axis].set_title(title)
        
        elif plotType == "bar": #Create a bar plot using the specified dataframe
            self.df.plot.bar(x, y, rot=rotation, legend=legendIndex, figsize=(size))
            plot.xlabel(xlabel)
            plot.ylabel(ylabel)
            plot.title(title)
        
        elif plotType == "pie": #Create a pie chart using the specified dataframe
            self.df.plot.pie(y=y, ax=axes[axis], labels=self.df[legendlabel], 
                             legend=legendIndex, autopct='%1.1f%%')
            axes[axis].yaxis.set_label_text(ylabel)
            axes[axis].set_title(title)      
    
    #This method is used to create stacked bar plots
    def stackedPlot(self, columns, rowAxis, colAxis, xlabel=None, ylabel=None, title=None, rotation=None):
        self.df[columns].plot(kind="bar", stacked=True, ax=axes[rowAxis,colAxis], rot=rotation)
        axes[rowAxis,colAxis].xaxis.set_label_text(xlabel)
        axes[rowAxis,colAxis].yaxis.set_label_text(ylabel)
        axes[rowAxis,colAxis].set_title(title)

In [ ]:
#Create an instance
user1 = Data()

#Run the userStates() method to make sure the user inputs a valid list
user1.userStates()

In [ ]:
#Create an empty list to append df_merged information we want to plot later
dummyList = []

#Create empty dataframes to append data from df_merged and COVID, respectively, based on user input
userMerged = Data(False) #Create an instance to pass through plotting methods later
userCOVID = Data(False)
i = Data(False) #Create an instance to pass through the userPlot() method

#Create the figure and set the axes for plotting the COVID data
fig, axes = plot.subplots(4, sharex=True, sharey=False, gridspec_kw={'hspace': 0}, figsize=(10,10))

#Iterate over every state from the user input
for state in userList:
    #Append a nested list of every state's information based on the user input
    dummyList.append(df_merged.loc[state].values[0:(len(df_merged.columns)+1)])
    #Creates a dummy dataframe based on each state
    i.df = covid.loc[covid['stateCode'] == state]
    #Plot the desired data with the userPlot method
    i.userPlot('line', "COVID-19 Plots by State", 'positive', "Cum. COVID-19 Cases", 
               True, x='date', xlabel="", axis=0, legendlabel='State')
    i.userPlot('line', "", 'death',"Cum. COVID-19 Deaths", False, x='date', 
               xlabel="", axis=1, legendlabel='State')
    i.userPlot('line', "", 'totalTestResults',"Cum. People Tested", False, x='date', 
               xlabel="", axis=2, legendlabel='State')
    i.userPlot('line', "", 'positiveIncrease',"Daily COVID-19 Cases", False, x='date', 
               xlabel="Date", axis=3, legendlabel='State')
    #Append the dummy dataframe to our userCOVID dataframe for every iteration of state
    userCOVID.df = userCOVID.df.append(i.df)

#Update the userMerged dataframe based on the dummyList of state's information
userMerged.df = pd.DataFrame(dummyList, columns = df_merged.columns).sort_values(by=['State'], ascending=True)

#Print results
print('Below is each state\'s political affiliation based on your input. \n')
print(userMerged.df[['State', 'govAffiliation', 'govName']].to_string(index=False))

In [ ]:
#Data wrangling on user input

#Overwrite the userCOVID dataframe to include only the max death, number of tests, and positive case values. 
#The values are cumulative so the max will correspond to the most recent date
userCOVID.df = pd.concat([userCOVID.df.groupby(['State'])['death'].max(), 
                          userCOVID.df.groupby(['State'])['positive'].max(),
                          userCOVID.df.groupby(['State'])['totalTestResults'].max()], 
                         axis=1)

#Calculate death percentage in people testing positive for COVID-19
userCOVID.df['deathPct'] = userCOVID.df['death'] / userCOVID.df['positive'] *100

#Calculate positive test percentage in people getting tested
userCOVID.df['posPct'] = userCOVID.df['positive'] / userCOVID.df['totalTestResults'] *100

In [ ]:
#Plot total COVID-19 metrics for the user input states

#Set figure and build subplots
fig, axes = plot.subplots(2, 2, figsize=(20,10))
fig.tight_layout(pad=10.0)

#Plot COVID-19 totals for the user input states using the stackedPlot method
userCOVID.stackedPlot(["positive","totalTestResults"],0,0, xlabel="", ylabel="Number of People", 
                      title="Positive COVID-19 Cases and Total Tests by State", rotation=45)
userCOVID.stackedPlot(["posPct"], 1, 0, xlabel="State", ylabel="Positive Tests (%)", 
                      title="Positive Test Rate from People Tested", rotation=45)
userCOVID.stackedPlot(["death", "positive"], 0, 1, xlabel="", ylabel="Number of People", 
                      title="Deaths and Positive COVID-19 Cases by State", rotation=45)
userCOVID.stackedPlot(["deathPct"], 1, 1, xlabel="State", ylabel="Deaths (%)", 
                      title="Death Rate Based on Positive COVID-19 Cases", rotation=45)

#Reformat the dataframe column values to look nicer when printed
format_dict = {'death':'{:,.0f}', 'positive':'{:,.0f}', 'totalTestResults':'{:,.0f}', 
               'deathPct':'{:.2f}%', 'posPct':'{:.2f}%'}
for key, value in format_dict.items():
    userCOVID.df[key] = userCOVID.df[key].apply(value.format)
    
#Print the table of data that are being plotted
print(userCOVID.df)
print("\n")

In [ ]:
#Plot HHS spending metrics for the user input states

#Create a bar plot for HHS funding by state
userMerged.userPlot("bar", "HHS Funding Received by State", 'Amount', 
                    "Funding Received ($)", False, x='State', xlabel="State", 
                    rotation=45, size=(10,5))

#Set figure and build subplots
fig, axes = plot.subplots(1,2, figsize=(10,10))
fig.tight_layout(pad=5.0)

#Create pie charts for HHS funding by state and fundering per capita by state
userMerged.userPlot("pie", "Total HHS Funding Received by State ($)", 'Amount', "", 
                    False, axis=0, legendlabel='State')
userMerged.userPlot("pie", "Total HHS Funding per Capita Received by State ($/person)", 
                    'perCap', "", False, axis=1, legendlabel='State')

In [ ]:
#Reformat the dataframe column values to look nicer when printed
userMerged.df['population2019'] = userMerged.df['population2019'].map('{:,.0f}'.format)
userMerged.df['Amount'] = userMerged.df['Amount'].map('${:,.0f}'.format)
userMerged.df['perCap'] = userMerged.df['perCap'].map('${:,.2f}'.format)

In [ ]:
#Print the table that is associated with the figures plotted above
print(userMerged.df[["govAffiliation", "govName", "State", "population2019", 
                     "Amount", "perCap"]].to_string(index=False))

### 4. Breakdown of HHS Spending

Data Decription:  The data in this section lists awards made by HHS (from February 2020 to June 2020) using emergency supplemental appropriation funding provided in the Coronavirus Preparedness and Response Supplemental Appropriations Act, 2020,the Families First Coronavirus Response Act, the CARES Act (Coronavirus Aid, Relief, and Economic Security Act), and the Paycheck Protection Program and Health Care Enhancement Act. 

Data Variables:

Number = The Catalog of Federal Domestic Assistance (CFDA) Number

Title = The title of the HHS Government Program

Name = The recipient of the government funds who provides COVID19 related Services

State = The State or U.S. Territory that received HHS COVID19 funding.

Amount = The Award Amount in dollars.

In [ ]:
#Merge the spendingHHS and abbreviations dataframes so we eliminate data from US territories
spendingHHS = pd.merge(left=abbreviations, right=spendingHHS, left_on='stateCode', right_on='stateCode')

In [ ]:
#Query 1:  What is the total amount of HHS health spending through June 30, 2020?
spendingHHS.Amount.sum()

In [ ]:
#Query 2:  What is the total amount of health spending by state through June 30, 2020?
spendingHHS.groupby('State').Amount.sum()

In [ ]:
#Query 3:  What is the total amount of health spending by a given state?
spendingHHS.loc[spendingHHS['stateCode'] == 'VA'].Amount.sum()

In [ ]:
#Show the full dataframe
pd.set_option("display.max_rows", None, "display.max_columns", None)

#Query 4: For each State and Program Title, what is the total amount of VA, NY, VT, TX, and CA spending?
stateSpending = (spendingHHS.loc[(spendingHHS['stateCode'] == 'VA') | (spendingHHS['stateCode'] == 'NY') | 
                                (spendingHHS['stateCode'] == 'VT') | (spendingHHS['stateCode'] == 'TX') | 
                                (spendingHHS['stateCode'] == 'CA')]
                 .groupby(['stateCode', 'Title'])
                 .Amount.sum()
                 .to_frame()
                 .sort_values(by=['stateCode','Amount'], ascending=[True, False]))
stateSpending

### 5. Treemap Visuals of HHS Spending

Resource/Source: Ryan Noonan, Python Treemaps, https://www.youtube.com/watch?v=eiq__4mg00o&t=393s

Spending Treemap charts were developed to representing data in the form of rectangles. The area or size of each rectangle corresponds to its numerical value.

In [ ]:
#Sort Program by Total Amount
Total_Amount_by_Program = (spendingHHS.groupby('Title')['Amount']
                      .sum()
                      .sort_values(ascending=False)
                      .reset_index(name='Amount'))

# Create a dataframe to include only award amounts greater than 100000000
Total_Amount_by_Program = Total_Amount_by_Program[Total_Amount_by_Program['Amount'] > 100000000]

In [ ]:
#Create a list of unique programs
Program = Total_Amount_by_Program.Title.unique()

In [ ]:
#Create Treemap settings, colors correspond to Program
custom_style = Style(legend_font_size=5)

Program_treemap = pygal.Treemap(width=300, height=300, style=custom_style, legend_at_bottom=True)
Program_treemap.title = 'COVID19 Spending - Total USA Amounts by HHS Program'

# Create a for loop for each of the Program types matching Program in the Program list variable, show total Amount
for i in Program:
    Program_treemap.add(i, Total_Amount_by_Program[Total_Amount_by_Program.Title == i]['Amount'])
#Display Treemap
Program_treemap

In [ ]:
#Reset index
stateSpending = stateSpending.reset_index()

In [ ]:
# Create a dataframe to include only award amounts greater than 10000000
stateSpending = stateSpending[stateSpending['Amount'] > 10000000]

In [ ]:
#Create treemap for VA spending
VA_Program = stateSpending.loc[stateSpending['stateCode'] == 'VA'].Title.unique()

custom_style = Style(legend_font_size=5)
VA_Program_treemap = pygal.Treemap(width=300, height=300, style=custom_style, legend_at_bottom=True)
VA_Program_treemap.title = 'COVID19 Spending - Total VA Amounts by HHS Program'

for j in VA_Program:
    VA_Program_treemap.add(j, stateSpending[stateSpending.Title == j]['Amount'].sum()) 

VA_Program_treemap

In [ ]:
#Create treemap for NY
NY_Program = stateSpending.loc[stateSpending['stateCode'] == 'NY'].Title

custom_style = Style(legend_font_size=5)
NY_Program_treemap = pygal.Treemap(width=300, height=300, style=custom_style, legend_at_bottom=True)
NY_Program_treemap.title = 'COVID19 Spending - Total NY Amounts by HHS Program'

for j in NY_Program:
    NY_Program_treemap.add(j, stateSpending[stateSpending.Title == j]['Amount'].sum()) 
    
NY_Program_treemap

In [ ]:
#Create treemap for VT
VT_Program = stateSpending.loc[stateSpending['stateCode'] == 'VT'].Title

custom_style = Style(legend_font_size=5)
VT_Program_treemap = pygal.Treemap(width=300, height=300, style=custom_style, legend_at_bottom=True)
VT_Program_treemap.title = 'COVID19 Spending - Total VT Amounts by HHS Program'

for j in VT_Program:
    VT_Program_treemap.add(j, stateSpending[stateSpending.Title == j]['Amount'].sum()) 
    
VT_Program_treemap

In [ ]:
#Create treemap for TX
TX_Program = stateSpending.loc[stateSpending['stateCode'] == 'TX'].Title

custom_style = Style(legend_font_size=5)
TX_Program_treemap = pygal.Treemap(width=300, height=300, style=custom_style, legend_at_bottom=True)
TX_Program_treemap.title = 'COVID19 Spending - Total TX Amounts by HHS Program'

for j in TX_Program:
    TX_Program_treemap.add(j, stateSpending[stateSpending.Title == j]['Amount'].sum()) 
    
TX_Program_treemap

In [ ]:
#Create treemap for CA
CA_Program = stateSpending.loc[stateSpending['stateCode'] == 'CA'].Title

custom_style = Style(legend_font_size=5)
CA_Program_treemap = pygal.Treemap(width=300, height=300, style=custom_style, legend_at_bottom=True)
CA_Program_treemap.title = 'COVID19 Spending - Total CA Amounts by HHS Program'

for j in CA_Program:
    CA_Program_treemap.add(j, stateSpending[stateSpending.Title == j]['Amount'].sum()) 
    
CA_Program_treemap